<a href="https://colab.research.google.com/github/shuyaguan/0826/blob/main/Shuya_Guan_HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1, use_residual=True):
        super(ConvBlock, self).__init__()
        self.use_residual = use_residual
        self.same_channels = (in_channels == out_channels)

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu1 = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, 1, padding)
        self.bn2 = nn.BatchNorm2d(out_channels)

        if use_residual and not self.same_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride, 0, bias=False),
                nn.BatchNorm2d(out_channels)
            )

        self.relu2 = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.use_residual:
            if not self.same_channels:
                identity = self.skip(identity)
            out += identity

        out = self.relu2(out)
        return out

class SpatialAttention(nn.Module):
    def __init__(self, in_channels):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, in_channels//8, kernel_size=1),
            nn.BatchNorm2d(in_channels//8),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels//8, 1, kernel_size=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        attention = self.conv(x)
        return x * attention

class AdvancedCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(AdvancedCNN, self).__init__()

        self.initial = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.block1 = nn.Sequential(
            ConvBlock(64, 64),
            ConvBlock(64, 64),
            SpatialAttention(64)
        )

        self.block2 = nn.Sequential(
            ConvBlock(64, 128, stride=2),
            ConvBlock(128, 128),
            SpatialAttention(128)
        )

        self.block3 = nn.Sequential(
            ConvBlock(128, 256, stride=2),
            ConvBlock(256, 256),
            ConvBlock(256, 256),
            SpatialAttention(256)
        )

        self.block4 = nn.Sequential(
            ConvBlock(256, 512, stride=2),
            ConvBlock(512, 512),
            ConvBlock(512, 512),
            SpatialAttention(512)
        )

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.initial(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
from tqdm.notebook import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects.double() / len(dataloader.dataset)

    return epoch_loss, epoch_acc

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects.double() / len(dataloader.dataset)

    return epoch_loss, epoch_acc

In [4]:
# 下载数据集
!wget http://madm.dfki.de/files/sentinel/EuroSAT.zip -O EuroSAT.zip

# 解压数据集
!unzip -q EuroSAT.zip -d 'EuroSAT/'

# 删除压缩文件
!rm EuroSAT.zip

# 确认数据集文件夹存在
!ls EuroSAT/

--2025-03-24 17:36:56--  http://madm.dfki.de/files/sentinel/EuroSAT.zip
Resolving madm.dfki.de (madm.dfki.de)... 131.246.195.183
Connecting to madm.dfki.de (madm.dfki.de)|131.246.195.183|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94280567 (90M) [application/zip]
Saving to: ‘EuroSAT.zip’

EuroSAT.zip         100%[===================>]  89.91M  20.0MB/s    in 4.5s    

2025-03-24 17:37:01 (20.0 MB/s) - ‘EuroSAT.zip’ saved [94280567/94280567]

2750


In [5]:
import torch
from torch.utils import data
from torchvision import datasets, transforms
import os

# 检查数据集路径
data_dir = './EuroSAT/2750/'
if not os.path.exists(data_dir):
    print(f"警告：路径 {data_dir} 不存在，尝试查找正确路径...")
    # 查找实际路径
    !find ./EuroSAT -type d -name "[0-9]*" | head -1
    # 检查EuroSAT目录结构
    !ls -la ./EuroSAT

# 定义数据转换
input_size = 224
imagenet_mean, imagenet_std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(input_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std)
])

val_transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std)
])

test_transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std)
])

# 修改这里的路径，可能需要根据实际情况调整
data_dir = './EuroSAT/'
dataset = datasets.ImageFolder(data_dir)

# 获取类别名称
class_names = dataset.classes
print("类别名称:", class_names)
print("类别总数:", len(class_names))

# 定义数据集类
class EuroSAT(data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        if self.transform:
            x = self.transform(self.dataset[index][0])
        else:
            x = self.dataset[index][0]
        y = self.dataset[index][1]
        return x, y

    def __len__(self):
        return len(self.dataset)

# 应用数据转换
train_data = EuroSAT(dataset, train_transform)
val_data = EuroSAT(dataset, val_transform)
test_data = EuroSAT(dataset, test_transform)

# 随机分割数据集
import numpy as np
train_size = 0.70
val_size = 0.15
indices = list(range(int(len(dataset))))
train_split = int(train_size * len(dataset))
val_split = int(val_size * len(dataset))
np.random.seed(42)
np.random.shuffle(indices)

train_data = data.Subset(train_data, indices=indices[:train_split])
val_data = data.Subset(val_data, indices=indices[train_split: train_split+val_split])
test_data = data.Subset(test_data, indices=indices[train_split+val_split:])
print("训练/验证/测试集大小: {}/{}/{}".format(len(train_data), len(val_data), len(test_data)))

# 创建数据加载器
batch_size = 32
num_workers = 2

train_loader = data.DataLoader(
    train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True
)
val_loader = data.DataLoader(
    val_data, batch_size=batch_size, num_workers=num_workers, shuffle=False
)
test_loader = data.DataLoader(
    test_data, batch_size=batch_size, num_workers=num_workers, shuffle=False
)

类别名称: ['2750']
类别总数: 1
训练/验证/测试集大小: 18900/4050/4050


In [6]:
import torch
from torch.utils import data
from torchvision import datasets, transforms

# 定义数据转换
input_size = 224
imagenet_mean, imagenet_std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(input_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std)
])

val_transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std)
])

test_transform = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean, imagenet_std)
])

# 定义数据集类
class EuroSAT(data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        if self.transform:
            x = self.transform(self.dataset[index][0])
        else:
            x = self.dataset[index][0]
        y = self.dataset[index][1]
        return x, y

    def __len__(self):
        return len(self.dataset)

# 加载数据集
data_dir = './EuroSAT/2750/'
dataset = datasets.ImageFolder(data_dir)

# 获取类别名称
class_names = dataset.classes
print("类别名称:", class_names)
print("类别总数:", len(class_names))

# 应用数据转换
train_data = EuroSAT(dataset, train_transform)
val_data = EuroSAT(dataset, val_transform)
test_data = EuroSAT(dataset, test_transform)

# 随机分割数据集
import numpy as np
train_size = 0.70
val_size = 0.15
indices = list(range(int(len(dataset))))
train_split = int(train_size * len(dataset))
val_split = int(val_size * len(dataset))
np.random.seed(42)
np.random.shuffle(indices)

train_data = data.Subset(train_data, indices=indices[:train_split])
val_data = data.Subset(val_data, indices=indices[train_split: train_split+val_split])
test_data = data.Subset(test_data, indices=indices[train_split+val_split:])
print("训练/验证/测试集大小: {}/{}/{}".format(len(train_data), len(val_data), len(test_data)))

# 创建数据加载器
batch_size = 32
num_workers = 2

train_loader = data.DataLoader(
    train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True
)
val_loader = data.DataLoader(
    val_data, batch_size=batch_size, num_workers=num_workers, shuffle=False
)
test_loader = data.DataLoader(
    test_data, batch_size=batch_size, num_workers=num_workers, shuffle=False
)

类别名称: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']
类别总数: 10
训练/验证/测试集大小: 18900/4050/4050


In [7]:
# 初始化模型
model = AdvancedCNN(num_classes=10)
model = model.to(device)

# 设置损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

# 训练参数
num_epochs = 30
best_val_acc = 0.0
best_model_wts = None

# 训练循环
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # 训练阶段
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

    # 验证阶段
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # 更新学习率
    scheduler.step(val_loss)

    # 保存最佳模型
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_wts = model.state_dict().copy()

    print("-" * 40)

# 加载最佳模型
model.load_state_dict(best_model_wts)
print(f"最佳验证集准确率: {best_val_acc:.4f}")

# 在测试集上评估
test_loss, test_acc = validate(model, test_loader, criterion, device)
print(f"测试集准确率: {test_acc:.4f}")

Epoch 1/30


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


  0%|          | 0/591 [00:00<?, ?it/s]

Train Loss: 1.4113, Train Acc: 0.4921


  0%|          | 0/127 [00:00<?, ?it/s]

...

Train Loss: 0.3085, Train Acc: 0.8983


  0%|          | 0/127 [00:00<?, ?it/s]

Val Loss: 0.1692, Val Acc: 0.9398
----------------------------------------
最佳验证集准确率: 0.9494


  0%|          | 0/127 [00:00<?, ?it/s]

测试集准确率: 0.9363


In [8]:
from torchvision import transforms
import torch

# 创建更高级的数据增强
advanced_train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),  # 添加随机旋转
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # 添加颜色抖动
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # 添加平移变换
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 更新训练数据集
train_data = EuroSAT(dataset, advanced_train_transform)
train_data = data.Subset(train_data, indices=indices[:train_split])
train_loader = data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)

In [10]:
from torchvision import transforms

# 创建更强大的数据增强
advanced_train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),  # 随机裁剪，保留70-100%的原图
    transforms.RandomHorizontalFlip(p=0.5),  # 50%几率水平翻转
    transforms.RandomVerticalFlip(p=0.5),    # 50%几率垂直翻转
    transforms.RandomRotation(30),           # 随机旋转30度以内
    transforms.ColorJitter(
        brightness=0.3,   # 亮度变化范围
        contrast=0.3,     # 对比度变化范围
        saturation=0.3,   # 饱和度变化范围
        hue=0.1           # 色调变化范围
    ),
    transforms.RandomAffine(
        degrees=0,
        translate=(0.1, 0.1),  # 在各方向平移10%以内
        scale=(0.9, 1.1)       # 缩放因子在0.9-1.1之间
    ),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [11]:
# 使用新的增强重新定义训练数据
train_data = EuroSAT(dataset, advanced_train_transform)
train_data = data.Subset(train_data, indices=indices[:train_split])

# 更新数据加载器
train_loader = data.DataLoader(
    train_data,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True
)

In [12]:
# 使用带标签平滑的损失函数
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # 0.1是标签平滑参数

In [13]:
# 初始化模型
model = AdvancedCNN(num_classes=10)
model = model.to(device)

# 设置优化器和学习率
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=5,
    verbose=True
)

# 训练参数
num_epochs = 50  # 增加训练轮数
best_val_acc = 0.0
best_model_wts = None

# 训练循环
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # 训练阶段
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")

    # 验证阶段
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # 更新学习率
    scheduler.step(val_loss)

    # 保存最佳模型
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_wts = model.state_dict().copy()
        # 连续10个epoch没有提升就提前停止
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= 10:
            print(f"早停：{epoch+1}轮后没有改进")
            break

    print("-" * 40)

# 加载最佳模型
model.load_state_dict(best_model_wts)
print(f"最佳验证集准确率: {best_val_acc:.4f}")

# 在测试集上评估
test_loss, test_acc = validate(model, test_loader, criterion, device)
print(f"测试集准确率: {test_acc:.4f}")

Epoch 1/50


  0%|          | 0/591 [00:00<?, ?it/s]

Train Loss: 1.7151, Train Acc: 0.4137


  0%|          | 0/127 [00:00<?, ?it/s]

Val Loss: 1.4045, Val Acc: 0.5825
----------------------------------------
Epoch 2/50


...

Train Loss: 0.6170, Train Acc: 0.9532


  0%|          | 0/127 [00:00<?, ?it/s]

Val Loss: 0.5996, Val Acc: 0.9573
----------------------------------------
最佳验证集准确率: 0.9647


  0%|          | 0/127 [00:00<?, ?it/s]

测试集准确率: 0.9504
